In [1]:
suppressMessages(library(car))
suppressMessages(library(effects))
suppressMessages(library(emmeans))

data(mtcars)

# Origin factor
mtcars$origin <- c('Japan','Japan','USA','USA','USA','USA','USA','Europe','Europe',
                   'Europe','Europe','Europe','Europe','Europe','USA','USA','USA',
                   'Europe','Japan','Japan','Japan','USA','USA','USA','USA',
                   'Europe','Europe','Europe','USA','Europe','Europe','Europe')
mtcars$origin <- as.factor(mtcars$origin)

# VS factor
vs.lab <- rep("",length(mtcars$vs)) 
vs.lab[mtcars$vs == 0] <- "V-shaped"
vs.lab[mtcars$vs == 1] <- "Straight"
mtcars$vs <- as.factor(vs.lab)

# Create fake interaction
mpg.fake          <- mtcars$mpg                  # copy mpg
mpg.idx           <- mtcars$origin == "Japan" &
                     mtcars$vs     == "V-shaped" # index of Japan-VShaped cell
mpg.fake[mpg.idx] <- mpg.fake[mpg.idx] + 15      # add constant to all data from that cell
mtcars$mpg.fake   <- mpg.fake   

mod <- lm(mpg.fake ~ origin + vs + origin:vs, data=mtcars)

# Advanced Uses of `emmeans`
In this final section of the lesson, we will examine more advanced ways we can use `emmeans`. Specifically, we will see how to make sure the follow-up tests match the Type II ANOVA tests, how to use `emmeans` to break-down more complex higher-order ANOVA effects and how to specify custom hypothesis tests using our own contrast weights.

## Type II Follow-up Tests
Returning back to the concept of omnibus tests, there is an important detail that we need to understand about the means generated by `emmeans`. To see this, let us go back to our original (non-fake) `mtcars` data where there is *no* 2-way interaction.

In [2]:
mod <- lm(mpg ~ origin + vs + origin:vs, data=mtcars)
print(Anova(mod))

Anova Table (Type II tests)

Response: mpg
          Sum Sq Df F value    Pr(>F)    
origin    179.61  2  5.2153 0.0124621 *  
vs        282.26  1 16.3921 0.0004117 ***
origin:vs   2.21  2  0.0641 0.9380730    
Residuals 447.70 26                      
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


On this basis, we may wish to follow-up the main effects using `emmeans`. Let us see what happens if we do this for `vs`

In [3]:
emm <- emmeans(mod, pairwise ~ vs)
print(emm$contrasts)

NOTE: Results may be misleading due to involvement in interactions



 contrast              estimate   SE df t.ratio p.value
 Straight - (V-shaped)      6.6 1.73 26   3.825  0.0007

Results are averaged over the levels of: origin 


Given that `vs` only has 2 levels, the follow-up test should be *identical* to the ANOVA test. However, the $p$-values are different. We should also have $t^2 = F$, but

In [4]:
t <- as.data.frame(emm$contrasts)$t.ratio[1]
print(t^2)

[1] 14.62853


So what is going on? The answer is that, in the presence of an interaction term, the means used by `emmeans` correspond to the Type III tests rather than the Type II. We can see this if we generate the Type III ANOVA table

In [5]:
mod.sum <- lm(mpg ~ origin + vs + origin:vs, data=mtcars, 
              contrasts=list(origin=contr.sum,vs=contr.sum))

# Type III ANOVA table
print(Anova(mod.sum, type="III"))

Anova Table (Type III tests)

Response: mpg
             Sum Sq Df  F value    Pr(>F)    
(Intercept) 10488.5  1 609.1096 < 2.2e-16 ***
origin        166.5  2   4.8354 0.0163901 *  
vs            251.9  1  14.6285 0.0007372 ***
origin:vs       2.2  2   0.0641 0.9380730    
Residuals     447.7 26                       
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


where we can see that the effect for `vs` now matches the result from `emmeans`. 

In order to get main effect follow-up tests that match the Type II tests, we need to provide the `submodel='type2'` argument.

In [6]:
emm <- emmeans(mod, pairwise ~ vs, submodel="type2")
print(emm$contrasts)

NOTE: Results may be misleading due to involvement in interactions



 contrast              estimate   SE df t.ratio p.value
 Straight - (V-shaped)     6.38 1.58 26   4.049  0.0004

Results are averaged over the levels of: origin 


Now the results match and the follow-up tests are calculated from a model where the interaction effect is assumed to be 0. We can also do the same with `origin`

In [7]:
emm <- emmeans(mod, pairwise ~ origin, submodel="type2", adjust="holm")
print(emm$contrasts)

NOTE: Results may be misleading due to involvement in interactions



 contrast       estimate   SE df t.ratio p.value
 Europe - Japan    -3.57 2.16 26  -1.651  0.1107
 Europe - USA       3.50 1.69 26   2.073  0.0964
 Japan - USA        7.07 2.26 26   3.127  0.0129

Results are averaged over the levels of: vs 
P value adjustment: holm method for 3 tests 


So just remember, when following-up main effects we need to provide this argument for consistency of inference.

## Higher-order Interactions


Although we have now seen how to follow-up a 2-way interaction with `emmeans`, things get more complex when we have even higher-order interactions in the model. In general, it is not recommended to go beyond a 3-way interaction due to the complexities that come with interpretation. You can, if you wish, but things start getting difficult very quickly. Here, we will demonstrate using `emmeans` to break-down a 3-way interaction.

For this demonstration, we will return to the `headache` dataset from the `datarium` package, as introduced in the workshop last week.

In [8]:
library('datarium')
data(headache)
print(head(headache))

# A data frame: 6 × 5
     id gender risk  treatment pain_score
* <int> <fct>  <fct> <fct>          <dbl>
1     1 male   low   X               79.3
2     2 male   low   X               76.8
3     3 male   low   X               70.8
4     4 male   low   X               81.2
5     5 male   low   X               75.1
6     6 male   low   X               73.1


We can proceed using our usual approach, though we will skip the call to `summary()` and the checking of assumptions (in the interests of saving space). This dataset is *balanced*, so the choice of sums-of-squares is immaterial.

In [9]:
mod <- lm(pain_score ~ gender*risk*treatment, data=headache)
print(Anova(mod))

Anova Table (Type II tests)

Response: pain_score
                       Sum Sq Df F value    Pr(>F)    
gender                 313.36  1 16.1957 0.0001625 ***
risk                  1793.56  1 92.6988   8.8e-14 ***
treatment              283.17  2  7.3177 0.0014328 ** 
gender:risk              2.73  1  0.1411 0.7084867    
gender:treatment       129.18  2  3.3384 0.0422001 *  
risk:treatment          27.60  2  0.7131 0.4942214    
gender:risk:treatment  286.60  2  7.4063 0.0013345 ** 
Residuals             1160.89 60                      
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


We can see that there is a significant 3-way interaction. How do we follow this up? We could adapt the `pairwise ~` syntax we used earlier to give

In [10]:
emm <- emmeans(mod, pairwise ~ gender:risk|treatment, adjust="holm")
print(emm$contrasts)

treatment = X:
 contrast                 estimate   SE df t.ratio p.value
 male high - female high     13.87 2.54 60   5.463  <.0001
 male high - male low        16.69 2.54 60   6.571  <.0001
 male high - female low      18.58 2.54 60   7.317  <.0001
 female high - male low       2.81 2.54 60   1.108  0.5448
 female high - female low     4.71 2.54 60   1.854  0.2059
 male low - female low        1.90 2.54 60   0.746  0.5448

treatment = Y:
 contrast                 estimate   SE df t.ratio p.value
 male high - female high      1.17 2.54 60   0.459  0.6477
 male high - male low         9.20 2.54 60   3.624  0.0024
 male high - female low      13.98 2.54 60   5.505  <.0001
 female high - male low       8.04 2.54 60   3.165  0.0073
 female high - female low    12.81 2.54 60   5.045  <.0001
 male low - female low        4.78 2.54 60   1.881  0.1297

treatment = Z:
 contrast                 estimate   SE df t.ratio p.value
 male high - female high     -1.35 2.54 60  -0.533  0.5958
 male hig

However, the problem here is that the `gender` and `risk` factors have been collapsed, leading to some comparisons that keep one factor constant (e.g. `male high - female high`) whereas others that change both factors (e.g. `male high - female low`).

A better approach is to get `emmeans` to more sensibly combine the levels of the various factors. In order to do this, we first use `emmeans()` to generate the means we want to compare. Then we use the `contrast()` function[^confunc-foot]] on those means to dictate how we want them to be compared. The `contrast()` function has two options, `interaction=` and `by=`, that allow us to specify these comparisons in a more sensible way. For instance, using the `interaction=` option and setting one factor to `'pairwise'` and the other to `'identity'`, we get the following

In [11]:
emm <- emmeans(mod, ~ gender*risk*treatment) # generate all cell means
contrast(emm, interaction=c(gender="pairwise", risk="identity"), 
         by="treatment", adjust="holm")

treatment = X:
 gender_pairwise risk_identity estimate   SE df t.ratio p.value
 male - female   high             13.87 2.54 60   5.463  <.0001
 male - female   low               1.90 2.54 60   0.746  0.4583

treatment = Y:
 gender_pairwise risk_identity estimate   SE df t.ratio p.value
 male - female   high              1.17 2.54 60   0.459  0.6477
 male - female   low               4.78 2.54 60   1.881  0.1297

treatment = Z:
 gender_pairwise risk_identity estimate   SE df t.ratio p.value
 male - female   high             -1.35 2.54 60  -0.533  0.5958
 male - female   low               4.68 2.54 60   1.841  0.1410

P value adjustment: holm method for 2 tests 

So we can see that there is a significant `gender` effect for treatment `X`, but only within the `high` risk group and not the `low` risk group.

## Custom Contrast Weights
As a final topic for this lesson, we will see how we can create custom hypotheses within `emmeans`. This is useful to know because sometimes it can be difficult to get `emmeans` to perform exactly the comparison you want using its various syntax options. Other times, you may be able to generate the tests you want, but only combined with other tests you do *not* want. In these instances, it becomes fiddly to then apply any multiple comparison correction. In both cases, we can have maximum control over the follow-up tests by providing `emmeans` with custom contrast weights. 

### Example for a 2-way Interaction
For the final time, let us go back to our fake 2-way interaction from `mtcars`. As an example, say that we have a significant 2-way interaction where we are interested in looking at `V-shaped` and `Straight` cars separately, but then comparing `Japan` to the *average* of `Europe` and `USA`. We can do this in `emmeans` using some custom contrast weights.

To begin, we will specify the model and then extract the means we are interested in using `emmeans`. This is important because the contrast weights are *relative to the means we have extracted*, not relative to the model parameters (as we saw earlier). As such, we need to know (a) what means the `emm` object contains, and (b) what *order* they are in

In [12]:
mod <- lm(mpg.fake ~ origin*vs, data=mtcars)
emm <- emmeans(mod, ~ origin*vs)

print(emm)

 origin vs       emmean   SE df lower.CL upper.CL
 Europe Straight   24.5 1.47 26     21.4     27.5
 Japan  Straight   28.6 2.40 26     23.7     33.5
 USA    Straight   20.8 2.40 26     15.8     25.7
 Europe V-shaped   18.3 1.69 26     14.8     21.7
 Japan  V-shaped   36.0 2.93 26     30.0     42.0
 USA    V-shaped   14.8 1.31 26     12.1     17.4

Confidence level used: 0.95 


So, now we know the means we have available and can specify the weights. The first 3 means correspond to `Straight` and so we can construct the weights for comparing `Japan` to the average of `Europe` and `USA` like so

In [13]:
EUR.Str.c <- c(1,0,0,0,0,0)
JAP.Str.c <- c(0,1,0,0,0,0)
USA.Str.c <- c(0,0,1,0,0,0)
EUR.V.c   <- c(0,0,0,1,0,0)
JAP.V.c   <- c(0,0,0,0,1,0)
USA.V.c   <- c(0,0,0,0,0,1)

c.1 <- JAP.Str.c - ((EUR.Str.c + USA.Str.c)/2)
print(c.1)

[1] -0.5  1.0 -0.5  0.0  0.0  0.0


We can then do the same thing with `V-shaped` engines

In [14]:
c.2 <- JAP.V.c - ((EUR.V.c + USA.V.c)/2)
print(c.2)

[1]  0.0  0.0  0.0 -0.5  1.0 -0.5


We can then pass both these sets of weights to the `contrast()` function to generate the tests. We use a list to pass multiple contrasts with custom names so we know what is being tested in the output. Notice we can also pass an `adjust=` method for multiple comparisons correction.

In [15]:
cons <- contrast(emm, adjust='holm', method=list('Straight: JAP-mean(EUR,USA)'=c.1,
                                                 'V-shaped: JAP-mean(EUR,USA)'=c.2))

print(cons)

 contrast                    estimate   SE df t.ratio p.value
 Straight: JAP-mean(EUR,USA)     5.99 2.78 26   2.155  0.0406
 V-shaped: JAP-mean(EUR,USA)    19.49 3.12 26   6.240  <.0001

P value adjustment: holm method for 2 tests 


So now we have a custom family of follow-up tests that produce exactly the comparisons that we want.

`````{topic} What do you now know?
In this section, we have explored some advanced topics around the use of the `emmeans` package to follow-up ANOVA models. After reading this section, you should have a good sense of:

- How to make sure any main effect follow-up tests are conducted using Type II means in `emmeans`.
- How these effects can be generated by specifying the correct additive models, but knowing that the tests will differ subtly.
- How to break-down a higher-order interaction using `emmeans`.
- How to use custom contrast weights to generate any comparisons we like, rather than relying on the `emmeans` syntax.

`````

[^confunc-foot]: Notice that this is called `contrast()` rather than `contrasts()` (with an `s` on the end). The latter is a different function in base `R`. If you are getting errors, make sure you have called the correct function. If in doubt, you can clarify your intention using `emmeans::contrast()`. If you get this wrong, the error message will say that `emmeans` does not contain a function called `contrasts()`.